# MA755 - Team Sandbox

Material from _Chapter 3. Classification_ of

>"[Hands-On Machine Learning with Scikit-Learn and TensorFlow](http://shop.oreilly.com/product/0636920052289.do)"<br>
by Aurélien Géron
Published by O'Reilly Media, Inc., 2017

Annotations, comments, and implementation on dataset by Angela Su, Tyler Miguel, Sebastian Bury

# Introduction

This is a College Scorecard dataset that gives various information about university and student level factors, such as college addmission, cost and finanaical aid, and student body demographics. This data is designed to "increase transparency" of "college costs and outcomes" to help students and families make informed decisions about the cost of furthering education. The data provided is aggregated at the institutional levels.

# Import Data

We start by downloading the dataset from online. To do this, we created a function to download the data from the US Department of Education website and extract the files from the zip file.

In [1]:
import os
import zipfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://ed-public-download.apps.cloud.gov/"
COLLEGE_PATH = "downloads"
COLLEGE_URL = DOWNLOAD_ROOT + COLLEGE_PATH + "/CollegeScorecard_Raw_Data.zip"
TO_PATH = "datasets/college"

def fetch_college_data(college_url=COLLEGE_URL, to_path=TO_PATH):
    os.makedirs(to_path, exist_ok=True)
    zip_path = os.path.join(to_path, "CollegeScorecard_Raw_Data.zip")
    urllib.request.urlretrieve(college_url, zip_path)
    college_zip = zipfile.ZipFile(zip_path)
    college_zip.extractall(path=to_path)
    college_zip.close()

In [ ]:
fetch_college_data() # only run if you don't already have the data local

The file `CollegeScorecard_Raw_Data.zip` is downloaded into the `datasets/college` directory. The unzipped `CollegeScorecard_Raw_Data` is created in the same directory. The file `MERGED2012_13_PP.csv`, which contains the most recent earnings data from College Scorecard data is stored in the `CollegeScorecard_Raw_Data` directory. 

We will load `MERGED2012_13_PP.csv` into our notebook. Additionally, we will load `MERGED2014_15_PP.csv` into our notebook so that we can extract key descriptive information about the universities that is not present in the `MERGED2012_13_PP.csv` file. 

In addition to loading the CSV into our notebook, we will do some data cleaning within our `load_college_data()` function to ensure the data structure of the features in the CSV file. Specifically, we noticed the file contained the string value "PrivacySuppressed" in many numeric fields. We converted these to `NaN` values. Furthermore, some of the string values in our dataframe were not rendering correct, so we explicitly stated the `dtype` of three columns, `ZIP`, `NPCURL`, and `ALIAS`.


In [2]:
import pandas as pd

def load_college_data(file, data_path=TO_PATH):
    csv_path = os.path.join(data_path, "CollegeScoreCard_Raw_Data",
                            file)
    # return Panda DataFrame with all data
    return pd.read_csv(csv_path, na_values = "PrivacySuppressed", 
                       dtype = {'ZIP':str,
                                'NPCURL':str,
                                'ALIAS':str})

Load the data from the `MERGED2012_13_PP.csv` and `MERGED2014_15_PP.csv` file into a `DataFrame` Pandas object stored in `college`. 

In [3]:
college_1213 = load_college_data(file='MERGED2012_13_PP.csv')
college_1415 = load_college_data(file='MERGED2014_15_PP.csv')
print(type(college_1213))
college_1213.head()

<class 'pandas.core.frame.DataFrame'>


,﻿UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,D100_L4,TRANS_4,DTRANS_4,TRANS_L4,DTRANS_L4,ICLEVEL,UGDS_MEN,UGDS_WOMEN,CDR3_DENOM,CDR2_DENOM
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,0.000000,1133.0,NaN,NaN,1,0.4879,0.5121,1405.0,1574.0
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,0.290236,1485.0,NaN,NaN,1,0.4200,0.5800,3153.0,3481.0
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,0.000000,1.0,NaN,NaN,1,0.4379,0.5621,264.0,336.0
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,0.307116,801.0,NaN,NaN,1,0.5425,0.4575,1208.0,1392.0
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,0.000000,1298.0,NaN,NaN,1,0.4046,0.5954,1951.0,1961.0


Each row is an institution with 1743 attributes as columns.

In [4]:
college_1213.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7793 entries, 0 to 7792
Columns: 1743 entries, ﻿UNITID to CDR2_DENOM
dtypes: float64(1724), int64(11), object(8)
memory usage: 103.6+ MB


This `.info()` method returns a brief description of the dataset, with each attribute from the columns and the type of the object. For example, we can see that the data in our college dataframe has 1724 float attributes, 11 integer attributes, and 8 object attributes (in our case, string or factor attributes).

# Variable Selection

Given the size of our dataset, we utilized the [College Data Documentation Report](http://https://collegescorecard.ed.gov/data/documentation/) to select our variables of interest. From this document, we were able to select variables of interest to answer our research question: how can we predict the Median income of a student 10 years out of college by using university factors. 


The variables we selected are:

University Description:
- INSTNM (institution name)
- CITY (the city of the school)
- STABBR (state postcode)
- ZIP (zip code)
- LATITUDE (latitude of location)
- LONGITUDE (longitude of location)

Features:
Factor variables
- ADM_RATE (admission rate as a percent)
--- turn into binaray (highly selective, selective, moderate, less than, non-selective
- MAIN (flag for main campus)
--- main branch
- ICLEVEL (highest level of award: 4-year, 2-year, less than 2-year)
--- level of school
- Control (Public, Private Non-profit, or Private For-Profit)
--- whether it is public or non
- HBCU= Historically black college
- PBI= Predominantly black college
- ANNHI= Alaska Native-/Native Hawaiian-serving Institutions
- TRIBAL= Tribal Colleges and Universities
- AANAPII= Asian American-/Native American-Pacific Islander-serving Institutions
- HSI= Hispanic-serving Institutions
- NANTI= Native American Non-Tribal-serving institution

Label data: 
- MN_EARN_WNE_P10 (Mean Earnings 10 years after college)

The university description variables were chosen to give a basic idea of the universty name and location, both the geographic latitude and longitude as well as city and zip in the state. The factor variables such as admission rate, level of award, and public/private can be correlated to see how these influence median income later for a student on in life. These basic factors were chosen since they give a profile of the school and could relate to later success, as do labels like "historically black college" and other racial indiciators. 

Due to the avaialable data in the `MERGED2012_13_PP.csv`, we had to load the `MERGED2014_15_PP.csv` file to extract longitude, latitude, HBCU, PBI, ANNHI, TRIBAL, AANAPII, HSI, and the NANT variables.

Below, we used both files to extract data and combine to what we will use moving forward. We will extract description data and the data we will use to explore relationships.

In [5]:
inst_1213 = college_1213.loc[:,['INSTNM','CITY','STABBR',
                                     'ZIP','ADM_RATE', 
                                     'MAIN','ICLEVEL',
                                     'CONTROL','MN_EARN_WNE_P10']]

inst_1415 = college_1415.loc[:, ['INSTNM', 'HBCU','PBI',
                                 'ANNHI', 'TRIBAL','AANAPII',
                                 'HSI', 'NANTI', 'LATITUDE','LONGITUDE']]

college = pd.merge(inst_1213, inst_1415, on=['INSTNM'])

Now that we have the variables we want, we will look take a look at our data set and check for missing values

# Remove missing data